## Tuning 3P Model with Random Search

In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams.update({'font.size': 20})

# Keras-related modules
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import keras_tuner as kt
from keras_tuner import HyperModel
from sklearn.model_selection import train_test_split

2022-07-22 23:58:50.600988: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


KeyboardInterrupt: 

### 1. Load data from file and prepare to be used for tuning

Load the EoS and the $M-R$ curves from processed 3P data files. Next, normalize the data and perform a train-test split as 80-20.

In [ ]:
# Load the un-normalized data
R = np.loadtxt('../data/mrc_td.txt')
P = np.loadtxt('../data/eos_td.txt')

# Normalize the data
r = (R - np.min(R)) / (np.max(R) - np.min(R))
p = (P - np.min(P)) / (np.max(P) - np.min(P))

# Perform train-test split as 80-20
x_tr, x_ts, y_tr, y_ts = train_test_split(r, p, test_size=0.2, shuffle=True, random_state=41)

### 2. Construct the search space

We seek to find the values of hyperparameters that optimizes the DNN for performance and accuracy. The choices of hyperparameters include number of layers, number of neurons/units in each layer, the activation functions on the inner layers and the output layer. 

Even with less number of hyperparameters, the search space can be large and the process may take a while to complete, depending on their ranges we choose. Therefore, to expedite the tuning, we fix some hyperparameters and search over the others.

In [ ]:
# Define the search space
class RegressionHyperModel(HyperModel):
    def __init__(self, input_shape):
        self.input_shape = input_shape    
        
    def build(self, hp):
        model = keras.Sequential()
        # Tune the number of layers
        for i in range(hp.Int('num_layers', 1, 4)):
            model.add(
                layers.Dense(
                    # Tune number of units separately
                    units=hp.Int(f"units_{i}", min_value=200, max_value=800, step=100),
                    activation='relu'
                )
            )
        model.add(layers.Dense(100, activation='linear'))
        model.compile(loss='mean_squared_error', optimizer='adam')
        return model

In [ ]:
# Initialize the input shape
input_shape = (x_tr.shape[1],)
hypermodel = RegressionHyperModel(input_shape)

### 3. Initialize search parameters

Set values of search parameters and create an early-stopping callback. 

In [ ]:
# Initialize the search
tuner = kt.RandomSearch(
    hypermodel,                # Pass the hypermodel object
    objective='val_loss',      # Quantity to monitor during tuning
    seed=42,                   # Set reproducibility of randomness
    max_trials=5000,           # Max number of trials with different hyperparameters
    executions_per_trial=1,    # Number of repeated trials with same hyperparameters
    directory="random_search", # Set directory to store search results
    project_name="td",         # Set the subdirectory name
    overwrite=True             # Choose if previous search results should be ignored
)
# Set up callback for early stopping 
stop_early = tf.keras.callbacks.EarlyStopping(monitor='loss', min_delta=1.0e-10, patience=10)

# Print the summary of search space
tuner.search_space_summary()

Now begin tuning the network.

In [ ]:
tuner.search(x_tr, y_tr, batch_size=32, epochs=1000, validation_data=(x_ts, y_ts), \
                callbacks=[stop_early], verbose=2)

### 4. Publish search results

Print the first few (5-10) top models and then pick the best model by hand.

In [ ]:
tuner.results_summary(num_trials=10)

In [ ]:
# Get the top model
models = tuner.get_best_models(num_models=10)
best_model = models[0]

# Build the best model
best_model.build(input_shape=(None, 100))

# Show the best model
best_model.summary()

Evaluate the best model and note the order of magnitude of the loss function for reference.

In [ ]:
loss = best_model.evaluate(x_ts, y_ts, verbose=0)
print("Loss = {:.4e}".format(loss))

### 5. Save the best model to file

Tuning a network is computationally expensive. Besides, the results are not reproducible because of the stochastic nature of this mechanism. Therefore, we tune a network only once for a given data set and do not repeat it unless either the input shape or the data set itself has changed.

Write the best model to file so that it can be loaded directly without repeating the search.

In [ ]:
best_model.save("../output/model_td.h5")